# Laporan Proyek Machine Learning - Leo Prangs Tobing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## Data Understanding

### Load Data

Siapkan semua library yang diperlukan proyek & load dataset. Dataset yang digunakan adalah **Book-Crossing: User review ratings** dari [Kaggle](https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset), dengan file yang digunakan adalah `Preprocessed_data.csv` (kombinasi informasi user, buku dan rating)

In [3]:
# # buka block kode ini, downgrage numpy==2.0.2 jadi versi yang kompatible dgn lingkunan scikit-suprise di GColab
# # restar ulang sesi, lalu comment kembali kode setelah selesai
# !pip install numpy==1.24.4 --force-reinstall

In [2]:
# !pip install -qq scikit-surprise==1.1.4
# !pip install -qq kagglehub

In [4]:
# load & EDA
import kagglehub
import pandas as pd

# Preprocesasing
import numpy as np # ajust dtype
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

# modelling
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader

# Untuk .py file (run in terminal)
from IPython.display import display

# Inferensi
import random

# Download latest version
path = kagglehub.dataset_download("ruchi798/bookcrossing-dataset")

print("Path to dataset files:", path)

df = pd.read_csv(f"{path}/Books Data with Category Language and Summary/Preprocessed_data.csv")
print("df.shape", df.shape)
display(df.head())

100%|██████████| 76.1M/76.1M [00:00<00:00, 200MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ruchi798/bookcrossing-dataset/versions/3
df.shape (1031175, 19)


,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
0,0,2,"stockton, california, usa",18.0000,0195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,Provides an introduction to classical myths pl...,en,['Social Science'],stockton,california,usa
1,1,8,"timmins, ontario, canada",34.7439,0002005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],timmins,ontario,canada
2,2,11400,"ottawa, ontario, canada",49.0000,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],ottawa,ontario,canada
3,3,11676,"n/a, n/a, n/a",34.7439,0002005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],NaN,NaN,NaN
4,4,41385,"sudbury, ontario, canada",34.7439,0002005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,"In a small town in Canada, Clara Callan reluct...",en,['Actresses'],sudbury,ontario,canada


**Insight:**

Terdapat **1.031.175 baris** data, dengan 19 kolom:

| No. | Kolom              | Deskripsi                                                                 |
|-----|--------------------|---------------------------------------------------------------------------|
| 1   | `Unnamed: 0`       | Indeks baris yang dihasilkan secara otomatis saat menyimpan file (bisa diabaikan). |
| 2   | `user_id`          | ID unik dari pengguna. Digunakan untuk mengidentifikasi user secara individual. |
| 3   | `location`         | Lokasi tempat tinggal pengguna dalam format "kota, provinsi, negara". |
| 4   | `age`              | Usia pengguna (dalam tahun). Tipe numerik. |
| 5   | `isbn`             | Nomor ISBN sebagai pengenal unik buku. |
| 6   | `rating`           | Nilai rating yang diberikan pengguna terhadap buku. Biasanya dalam skala 0–10. |
| 7   | `book_title`       | Judul lengkap dari buku. |
| 8   | `book_author`      | Nama penulis buku. |
| 9   | `year_of_publication` | Tahun buku tersebut diterbitkan. |
| 10  | `publisher`        | Nama penerbit buku. |
| 11  | `img_s`            | URL ke gambar sampul buku berukuran kecil (small). |
| 12  | `img_m`            | URL ke gambar sampul buku berukuran sedang (medium). |
| 13  | `img_l`            | URL ke gambar sampul buku berukuran besar (large). |
| 14  | `Summary`          | Ringkasan atau deskripsi isi buku. Dapat digunakan sebagai fitur teks dalam sistem rekomendasi. |
| 15  | `Language`         | Bahasa yang digunakan dalam buku (contoh: `en` untuk English). |
| 16  | `Category`         | Kategori atau genre buku, biasanya dalam bentuk list string (misalnya: `['Social Science']`). |
| 17  | `city`             | Kota asal pengguna (dipecah dari kolom `location`). |
| 18  | `state`            | Negara bagian atau provinsi asal pengguna (dipecah dari kolom `location`). |
| 19  | `country`          | Negara asal pengguna (dipecah dari kolom `location`). |

### EDA
Beberapa analisis yang punya insight penting (ditampilkan di markdown setelah output kode di bawah).

In [5]:
# Melihat jumlah user dan jumlah buku
print(f"{df['user_id'].nunique()} user, {df['isbn'].nunique()} buku")
print('\n---------- Change DType ---------')
display(df.info())
display(df[df['age'] == 5].head(3))
display(df[df['year_of_publication'] == 1376])

print('\n--------- Null Values ---------')
print('df.isna().sum():\n',df.isna().sum())

print('\n--------- Outliers ---------')
display('df.describe().T: ',df.describe().T)


print('\n--------- Invalid Values ---------')
for col in ['age', 'Language', 'Category']: # kolom kategori atau ordinal yang perlu diperiksa
    print(f"{col}: {df[col].unique().tolist()}")
display(df[df['Summary'] == '9'].head(3))

92107 user, 270170 buku

---------- Change DType ---------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031175 entries, 0 to 1031174
Data columns (total 19 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   Unnamed: 0           1031175 non-null  int64  
 1   user_id              1031175 non-null  int64  
 2   location             1031175 non-null  object 
 3   age                  1031175 non-null  float64
 4   isbn                 1031175 non-null  object 
 5   rating               1031175 non-null  int64  
 6   book_title           1031175 non-null  object 
 7   book_author          1031174 non-null  object 
 8   year_of_publication  1031175 non-null  float64
 9   publisher            1031175 non-null  object 
 10  img_s                1031175 non-null  object 
 11  img_m                1031175 non-null  object 
 12  img_l                1031175 non-null  object 
 13  Summary              1031175 non-null  obje

None

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
63237,63237,161619,"lisboa, n/a, portugal",5.0,0446672211,0,Where the Heart Is (Oprah's Book Club (Paperba...,Billie Letts,1998.0,Warner Books,http://images.amazon.com/images/P/0446672211.0...,http://images.amazon.com/images/P/0446672211.0...,http://images.amazon.com/images/P/0446672211.0...,9,9,9,lisboa,NaN,portugal
97731,97731,119749,"omaha, nebraska, usa",5.0,0307132668,0,"Bye, Bye, Butterfree (Pokemon Adventure (Golde...",Diane Muldrow,1999.0,Golden Books,http://images.amazon.com/images/P/0307132668.0...,http://images.amazon.com/images/P/0307132668.0...,http://images.amazon.com/images/P/0307132668.0...,"After helping Butterfree defeat Team Rocket, A...",en,['Juvenile Fiction'],omaha,nebraska,usa
137322,137322,119749,"omaha, nebraska, usa",5.0,0060256664,10,The Giving Tree,Shel Silverstein,1964.0,HarperCollins,http://images.amazon.com/images/P/0060256664.0...,http://images.amazon.com/images/P/0060256664.0...,http://images.amazon.com/images/P/0060256664.0...,9,9,9,omaha,nebraska,usa


,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
946880,946880,170186,"tehran, n/a, iran",27.0,964442011X,4,Tasht-i khun,IsmaÂ°il Fasih,1376.0,Nashr-i Alburz,http://images.amazon.com/images/P/964442011X.0...,http://images.amazon.com/images/P/964442011X.0...,http://images.amazon.com/images/P/964442011X.0...,9,9,9,tehran,NaN,iran



--------- Null Values ---------
df.isna().sum():
 Unnamed: 0                 0
user_id                    0
location                   0
age                        0
isbn                       0
rating                     0
book_title                 0
book_author                1
year_of_publication        0
publisher                  0
img_s                      0
img_m                      0
img_l                      0
Summary                    0
Language                   0
Category                   0
city                   14103
state                  22798
country                35374
dtype: int64

--------- Outliers ---------


'df.describe().T: '

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,1031175.0,515587.000000,297674.726251,0.0,257793.5,515587.0000,773380.5,1031174.0
user_id,1031175.0,140594.373956,80524.435020,2.0,70415.0,141210.0000,211426.0,278854.0
age,1031175.0,36.429017,10.353539,5.0,31.0,34.7439,41.0,99.0
rating,1031175.0,2.839022,3.854149,0.0,0.0,0.0000,7.0,10.0
year_of_publication,1031175.0,1995.282684,7.309340,1376.0,1992.0,1997.0000,2001.0,2008.0



--------- Invalid Values ---------
age: [18.0, 34.74389988072476, 49.0, 30.0, 36.0, 29.0, 60.0, 27.0, 71.0, 40.0, 53.0, 65.0, 46.0, 47.0, 26.0, 56.0, 37.0, 39.0, 25.0, 31.0, 41.0, 20.0, 58.0, 42.0, 38.0, 52.0, 21.0, 22.0, 34.0, 33.0, 19.0, 57.0, 51.0, 35.0, 32.0, 62.0, 45.0, 74.0, 44.0, 43.0, 68.0, 54.0, 55.0, 24.0, 59.0, 28.0, 70.0, 17.0, 23.0, 48.0, 14.0, 50.0, 16.0, 61.0, 15.0, 66.0, 67.0, 84.0, 82.0, 9.0, 72.0, 81.0, 64.0, 75.0, 13.0, 76.0, 69.0, 73.0, 63.0, 77.0, 79.0, 7.0, 12.0, 90.0, 80.0, 85.0, 78.0, 83.0, 8.0, 93.0, 11.0, 10.0, 97.0, 5.0, 86.0, 92.0, 99.0, 96.0, 94.0, 89.0, 6.0, 95.0, 98.0]
Language: ['en', '9', 'de', 'fr', 'es', 'ca', 'it', 'da', 'nl', 'pt', 'gl', 'ro', 'no', 'el', 'ms', 'la', 'ru', 'zh-CN', 'ga', 'pl', 'tl', 'th', 'ja', 'cy', 'fa', 'eo', 'gd', 'hi', 'vi', 'ar', 'zh-TW', 'ko', 'sv']
Category: ["['Social Science']", "['Actresses']", "['1940-1949']", "['Medical']", "['Design']", "['Fiction']", "['History']", '9', "['Nature']", "['Humor']", "['Cooking']", "['Re

,Unnamed: 0,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_s,img_m,img_l,Summary,Language,Category,city,state,country
68,68,8,"timmins, ontario, canada",34.7439,0671870432,0,PLEADING GUILTY,Scott Turow,1993.0,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,9,9,9,timmins,ontario,canada
69,69,11676,"n/a, n/a, n/a",34.7439,0671870432,8,PLEADING GUILTY,Scott Turow,1993.0,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,9,9,9,NaN,NaN,NaN
70,70,24539,"cabrils, catalunya, spain",34.7439,0671870432,0,PLEADING GUILTY,Scott Turow,1993.0,Audioworks,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,http://images.amazon.com/images/P/0671870432.0...,9,9,9,cabrils,catalunya,spain


**Insight:**
- 92.107 user, 270.170 buku
- **change Dtype:** `age` & `year_of_publication` dapat diubah ke **int**
- **null value:** `book_author` = 1, `city` = 14k, `state` = 22k, `country` = 35k
- **number distribution:**
  - **outliers**: `age` diusia 5 untuk rata-rata pembaca usia 36 tahun, dan `year_of_publication` yang punya tahun 1376 untuk rata rata tahun 1995.
  - 50% data `rating` bernilai 0, mungkin karena: Rating default (belum memberikan penilaian) atau pengguna tidak suka bukunya.
- **Invalid Value**:
  - nilai `34.74389988072476` pade `age` perlu dibulatkan
  - nilai `9` pada `Summary`, `Language`, dan `Category` bisa berarti placeholder untuk metadata buku yang tidak perlu ditampilkan kembali setelah kemunculan pertama

Catatan: Hasil insight hanya untuk pemahaman umum, berguna atau tidak tergantung apakah kolom dipakai untuk modelling.

## Data Preparation

### Pembersihan
Hapus kolom tidak berguna, ubah tipe data age & year menjadi **int**, isi null menggunakan string kosong (""), & ubah data invalid "9" menjadi string kosong ("").

In [6]:
df2 = df.copy(deep=True) # Berganti ke versi 2 (lebih bersih)

# --- CLEANING ---
# Hapus 7 kolom yang tidak diperlukan
df2 = df2.drop(columns=['Unnamed: 0', 'img_s', 'img_m', 'img_l', 'city', 'state', 'country', 'Summary'])

# ubah tipe data
df2['age'] = df2['age'].astype(int)
df2['year_of_publication'] = df2['year_of_publication'].astype(int)

# isi nilai null
df2['book_author'] = df2['book_author'].astype(str).fillna('')

# ubah data invalid
df2['Language'] = df2['Language'].replace('9', '', regex=False)
df2['Category'] = df2['Category'].replace('9', '', regex=False)

df2.shape

(1031175, 11)

- `df2` digunakan untuk preprocessing 2 model selanjutnya yang berdasarkan unik book (`df_book`) untuk model CBF, data unik relasi user-book-rating (`train_df_cf`) untuk model CF, dan `test_ground_truth` untuk evaluasi.
- Walau dapat dipakai untuk model CBF, Kolom `Summary` dihapus karena dapat menimbulkan noise.

### Sampling
Ambil irisan dari masing-masing top 500 User dan Buku (untuk performa).

In [7]:
# --- SAMPLING ---
# ambil 500 user_id yang paling sering muncul
top_500_users = df['user_id'].value_counts().nlargest(500).index.tolist()

# ambil 500 isbn yang paling sering muncul
top_500_isbns = df['isbn'].value_counts().nlargest(500).index.tolist()

print("Top 500 User IDs:")
print(top_500_users[:10]) # print beberapa contoh
print("\nTop 500 ISBNs:")
print(top_500_isbns[:10]) # print beberapa contoh

# Filter the DataFrame to include only interactions from the top 500 users
df_filtered_users = df2[df2['user_id'].isin(top_500_users)]

# Further filter the result to include only interactions with the top 500 ISBNs
df2 = df_filtered_users[df_filtered_users['isbn'].isin(top_500_isbns)].reset_index(drop=True)

df2.shape

Top 500 User IDs:
[11676, 198711, 153662, 98391, 35859, 212898, 278418, 76352, 110973, 235105]

Top 500 ISBNs:
['0971880107', '0316666343', '0385504209', '0060928336', '0312195516', '044023722X', '0142001740', '067976402X', '0671027360', '0446672211']


(29745, 11)

Dataset hasil filter: **~29.000** baris.

### Pembuatan Data untuk Model


#### CBF: `df_book`, `tfidf_matrix`, `isbn_to_index` & `index_to_isbn`
Versi unik buku + combined_features (gabungan title, author, publisher, language, category) untuk CBF. Setiap fitur penting digabung karena TF-IDF Butuh Representasi Teks Tunggal. `df_book` dipakai juga saat menampilkan Top-N rekomendasi

In [8]:
df_book = df2.copy(deep=True)

# ambil nilai book unik berdasarka isbn
df_book = df_book.drop_duplicates('isbn')

# Gabungkan kolom-kolom teks (nilai pengelompokkan dan kepemilikan)
df_book['combined_features'] = df_book[['book_title', 'book_author', 'publisher', 'Language', 'Category']].agg(' '.join, axis=1)

pd.set_option('display.max_colwidth', None) # Jangan potong isi kolom
pd.set_option('display.width', None) # Biarkan lebar menyesuaikan layar
pd.set_option('display.max_columns', None) # Tampilkan semua kolom jika banyak

# sederhanakan tabel
df_book = df_book[['isbn', 'book_title', 'combined_features']].reset_index(drop=True)

print(df_book.shape)
df_book.sample(10)

(500, 3)


,isbn,book_title,combined_features
470,0440224594,The Return Journey,The Return Journey MAEVE BINCHY Dell en ['Fiction']
213,0375702709,A Lesson Before Dying (Vintage Contemporaries (Paperback)),A Lesson Before Dying (Vintage Contemporaries (Paperback)) Ernest J. Gaines Vintage Books USA en ['African American men']
171,038533334X,Charming Billy,Charming Billy Alice McDermott Delta
374,0425116840,The Cardinal of the Kremlin (Jack Ryan Novels),The Cardinal of the Kremlin (Jack Ryan Novels) Tom Clancy Berkley Publishing Group en ['Fiction']
438,0312983271,Full House (Janet Evanovich's Full Series),Full House (Janet Evanovich's Full Series) Janet Evanovich St. Martin's Paperbacks
211,0385335881,Shopaholic Takes Manhattan (Summer Display Opportunity),Shopaholic Takes Manhattan (Summer Display Opportunity) Sophie Kinsella Delta en ['Fiction']
230,0743418174,Good in Bed,Good in Bed Jennifer Weiner Washington Square Press en ['Fiction']
369,0440235596,Tara Road,Tara Road Maeve Binchy Dell Publishing Company en ['Fiction']
496,0515128546,Tears of the Moon (Irish Trilogy),Tears of the Moon (Irish Trilogy) Nora Roberts Jove Books en ['Fiction']
25,0671004573,Before I Say Good-Bye,Before I Say Good-Bye Mary Higgins Clark Pocket


**Proses setelah fitur digabung jadi 1 kalimat:**
- Membandingkan kemiripan antar buku berdasarkan konten atau metadata-nya. Caranya? Fitur-fitur dalam string (`combined_features`)  ditransformasikan ke dalam bentuk vektor menggunakan **TF-IDF (Term Frequency - Inverse Document Frequency)**. TF-IDF mengubah kumpulan teks menjadi representasi vektor numerik yang menekankan kata-kata penting dan khas, lalu digunakan untuk mengukur kemiripan antar dokumen.
- **Hasilnya:** Didapatkan matriks TF-IDF dengan ukuran 500 ✖ 230

In [9]:
# TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Transformasikan ke bentuk numerik
tfidf_matrix = tfidf.fit_transform(df_book['combined_features'])

# Ukuran matriks (baris: isbn, kolom: kata unik dari combined_features)
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

# Contoh kata-kata (fitur) yang dihasilkan
feature_names = tfidf.get_feature_names_out()
print(f"Contoh fitur: {feature_names[:20]}")

TF-IDF matrix shape: (500, 1377)
Contoh fitur: ['12' '1984' '1st' '20th' '21st' '2nd' '45' '451' 'abortion' 'absolute'
 'accident' 'according' 'account' 'ace' 'acres' 'action' 'adams' 'adriana'
 'adultery' 'adventure']


Membuat index `isbn_to_index` dan `index_to_isbn`: digunakan untuk menjembatani antara format ISBN (string) dan posisi data dalam matriks TF-IDF (indeks numerik) sehingga memungkinkan pencarian dan interpretasi kemiripan antar buku.

In [31]:
isbn_to_index = {isbn: idx for idx, isbn in enumerate(df_book['isbn'])}
index_to_isbn = {v: k for k, v in isbn_to_index.items()}
print(isbn_to_index)
print(index_to_isbn)

{'0440234743': 0, '0452264464': 1, '0971880107': 2, '0345402871': 3, '0345417623': 4, '0446310786': 5, '0449005615': 6, '0671888587': 7, '0553582747': 8, '0425182908': 9, '0842342702': 10, '0440225701': 11, '042511774X': 12, '0804106304': 13, '0140067477': 14, '0345465083': 15, '0316769487': 16, '0679429220': 17, '0671867156': 18, '0451166892': 19, '0786868716': 20, '0446612545': 21, '0671027360': 22, '0671042858': 23, '0060976845': 24, '0671004573': 25, '038572179X': 26, '0330332775': 27, '0060915544': 28, '0060977493': 29, '0316096199': 30, '0316601950': 31, '0316666343': 32, '0316776963': 33, '0316899984': 34, '0345378490': 35, '0375705856': 36, '0375727345': 37, '0380717018': 38, '0380718332': 39, '0385497466': 40, '0385508042': 41, '0385511612': 42, '0385720106': 43, '0385730586': 44, '0425163407': 45, '0440224764': 46, '0440226430': 47, '044023722X': 48, '0446364800': 49, '0446605484': 50, '0446610399': 51, '0446612790': 52, '0452282152': 53, '0515130389': 54, '0671027387': 55, '

**Kenapa Diperlukan:**
- Matriks `tfidf_matrix` adalah array numerik (baris ke-0, ke-1, dst.).
- Untuk mencari cosine similarity antar buku, perlu tahu indeks baris dari sebuah ISBN.

#### `train_data` & `test_ground_truth`
- Buat ground truth: Ambil hanya buku dengan rating >= 5.
- Pisahkan test dan train: Gunakan 1 buku terakhir dari user sebagai test, sisanya sebagai train.
- Model Content-Based Filtering (CBF) yang menggunakan kemiripan antar fitur dievaluasi menggunakan data train dan test ini.

In [11]:
# --- Filter rating >= 5 (threshold minimum rating yang dianggap relevan (Disukai)) ---
liked = df2[df2["rating"] >= 5]

# --- Mapping user -> daftar (isbn, rating) ---
user_liked_books = defaultdict(list)

for _, row in liked.iterrows():
    user_liked_books[row["user_id"]].append((row["isbn"], row["rating"]))

# --- Split ke train dan test (test: buku dengan rating tertinggi) ---
train_data = []
test_ground_truth = {}

for user, books in user_liked_books.items():
    if len(books) < 2:
        print("ada user dengan data terlalu sedikit di-skip")
        continue  # user dengan data terlalu sedikit di-skip

    # Urutkan buku berdasarkan rating (tinggi ke rendah)
    books_sorted = sorted(books, key=lambda x: x[1], reverse=True)
    highest_rating = books_sorted[0][1]

    # Ambil semua buku dengan rating tertinggi sebagai test
    test_books = [isbn for isbn, rating in books_sorted if rating == highest_rating]

    # Sisanya masuk ke data latih
    train_books = [isbn for isbn, rating in books_sorted if rating < highest_rating]

    if len(test_books) > 0 and len(train_books) > 0:
        test_ground_truth[user] = test_books
        train_data.extend([(user, isbn) for isbn in train_books])

print(user_liked_books)

ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user dengan data terlalu sedikit di-skip
ada user d

`test_ground_truth` menunjukkan data kebenaran (buku yang seharusnya disukai/dibaca oleh pengguna di data uji. Jika kita lihat hasilnya

In [12]:
print(train_data[:5])
print(test_ground_truth)

[(7346, '0440234743'), (7346, '0804106304'), (7346, '0345361792'), (7346, '0345380371'), (7346, '0440221471')]
{7346: ['0440225701', '0590353403'], 11676: ['0345417623', '0449005615', '0671042858', '0671004573', '0316096199', '0385511612', '0446610399', '0671027387', '1558744150', '014100018X', '0316569321', '0439064864', '0451526341', '051513287X', '067976402X', '0743225406', '0312195516', '1573225789', '0060938455', '0553578693', '0446672211', '0449212602', '0385335482', '0671028375', '0345361792', '014025448X', '0425158616', '0062502182', '0060175400', '034540288X', '0380718340', '0446610038', '0553280341', '0553375407', '059035342X', '074343627X', '0446604666', '0142004235', '0385474016', '0842329218', '0425180964', '0439139600', '0553272535', '0440998050', '0767905385', '0553268880', '0060199652', '0385479565', '0515132020', '038079487X', '0452282829', '044651862X', '0671001795', '0609804138', '0679745203', '038550120X', '0553280368', '0515126772', '0743418204', '1558745157', '034

Hasil memperlihatkan target `test_ground_truth` memiliki jumlah bervariasi karena diambil dari list kelompok buku dengan rating tertinggi


#### `trainset_surp_cf` (CF)
- Dibuat menggunakan `train_data`, yang kemudian dipakai untuk melatih model Collaborative Filtering.
- Model CF yang menggunakan pola user book dari user lain yang serupa (dari train data) dan merekomendasikannya, hasilnya akan dievaluasi dengan data testing.

In [33]:
# Buat DataFrame dari train_data
train_df_cf = pd.DataFrame(train_data, columns=["user_id", "isbn"])
train_df_cf["rating"] = 5  # Karena semua data ini adalah rating >= 5 (menyederhanakan model)

**Dengan memberikan rating 5:**
- Dianggap bahwa setiap interaksi adalah positif (user menyukai buku tersebut).
- Ini umum dalam skenario implicit feedback, untuk mensimulasikan preferensi tinggi.

**Proses:** Data diubah ke format surprise dengan skala rating 1–5 agar bisa digunakan untuk pelatihan model rekomendasi.

In [34]:
# Buat dataset surprise
reader = Reader(rating_scale=(1, 5)) # memberi tahu bahwa semua rating berada pada skala 1 hingga 5
train_data_surp = Dataset.load_from_df(train_df_cf[["user_id", "isbn", "rating"]], reader) # menjadi objek dataset yang bisa digunakan oleh library surprise
trainset_surp_cf = train_data_surp.build_full_trainset() # trainset adalah objek yang berisi semua data pelatihan yang siap digunakan oleh algoritma surprise

**Hasil:** Dihasilkan trainset siap pakai untuk model CF (seperti SVD).

## Modelling

### Content-Based Filtering (CBF)
Membuat sistem CBF untuk merekomendasikan buku berdasarkan kemiripan konten (TF-IDF) dari buku-buku yang pernah disukai oleh pengguna. Caranya:
- Menggunakan Cosine Similarity untuk mengukur kemiripan arah antara dua vektor dalam ruang vektor, dengan nilai berkisar dari -1 hingga 1
- Ambil semua buku yang disukai,
- Untuk setiap buku, ambil 5 buku paling mirip,
- Gabungkan semua rekomendasi dari buku-buku yang disukai pengguna,
- Gunakan dict.fromkeys untuk menghapus duplikat sambil mempertahankan urutan,
- Ambil hanya 10 buku teratas sebagai hasil akhir.

In [14]:
def get_similar_books(isbn, top_n=10):
    if isbn not in isbn_to_index:
        return []
    idx = isbn_to_index[isbn]
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    similar_indices = cosine_sim.argsort()[::-1][1:top_n+1]
    return [index_to_isbn[i] for i in similar_indices]

# Ambil semua buku yang disukai user dari train
user_liked_books = defaultdict(list)
for user, isbn in train_data: # Data Uji
    user_liked_books[user].append(isbn)

# Buat prediksi rekomendasi berdasarkan kesamaan buku yang disukai
predictions_cbf = {}
for user, liked_books in user_liked_books.items():
    recs = []
    for b in liked_books:
        recs.extend(get_similar_books(b, top_n=5))  # bisa ubah top_n
    # Filter duplikat
    predictions_cbf[user] = list(dict.fromkeys(recs))[:10]

print("predictions_cbf (CBF):", predictions_cbf)
print("Target Ground Truth:", test_ground_truth)

predictions_cbf (CBF): {7346: ['044022165X', '0440241537', '0440211727', '0440220602', '0440241073', '0804114986', '080411109X', '080410753X', '0671021001', '0446672211'], 11676: ['044022165X', '0440241537', '0440211727', '0440220602', '0440241073', '0842329129', '0842329218', '0842329242', '0425167313', '0451166892'], 13552: ['0385484518', '0671004565', '0553582755', '0786885688', '0515132020', '0440220602', '0440241073', '0440213525', '044023722X', '044021145X'], 30533: ['0380789019', '0380789035', '1573229326', '0441569595', '0743418174', '0375707972', '0679745203', '0375703861', '0380002930', '0375702709'], 31315: ['0446612545', '0375705856', '0553273914', '038550120X', '0375702709', '0553292722', '0553561618', '055356451X', '0553579606', '0553571885'], 78973: ['0553579754', '0553580221', '0553582755', '042513525X', '0345386108', '0670892963', '014028009X', '0141000198', '0330332775', '0140119906'], 101209: ['044022165X', '0440241537', '0440211727', '0440220602', '0440241073', '037

- `predictions_cbf` Ini adalah top-10 daftar ISBN buku yang direkomendasikan untuk tiap pengguna, berdasarkan konten buku yang mirip dengan yang pernah disukai.
- Hasil Prediction model CBF dan Target Ground Truth akan dibandingkan untuk mendapatkan skor evaluasi

### Collaborative Filtering (CF) – SVD
- Berdasarkan pola rating yang diberikan oleh pengguna lain.
- Menggunakan **SVD (Singular Value Decomposition)** dari library `surprise` untuk memfaktorkan matriks user-item menjadi representasi laten. Cara Kerjanya:
 - SVD adalah teknik dari aljabar linier untuk memfaktorkan matriks menjadi:
 $$
  R \approx U \cdot \Sigma \cdot V^T
  $$
  - `R`: matriks user-item (rating), `U`: representasi pengguna dalam ruang laten, `V`: representasi item (buku) dalam ruang laten, `Σ`: bobot (singular values).
  - Tidak peduli isi/konten buku → murni berdasarkan pola interaksi. Contoh: "User A menyukai buku X dan Y, maka kemungkinan besar juga akan suka Z."

In [35]:
model_cf = SVD()
model_cf.fit(trainset_surp_cf)

**Hasil:** Proses ini menghasilkan model Collaborative Filtering berbasis SVD yang telah siap digunakan untuk memberikan prediksi rating dan rekomendasi buku, meskipun data awal hanya berupa interaksi (tanpa rating eksplisit).

**Penjelasan Kode dibawah:**
- **Bangun Rekomendasi untuk tiap user**
- Tujuan Fungsi `get_top_n_cf(...)` Membuat sistem rekomendasi berbasis Collaborative Filtering (CF) menggunakan model SVD untuk memprediksi top-N buku yang mungkin disukai oleh setiap user.

In [36]:
def get_top_n_cf(model, trainset, all_isbns, users, n=10):
    top_n = defaultdict(list)

    for user in users:
        try:
            inner_uid = trainset.to_inner_uid(user)
        except ValueError:
            continue  # user tidak dikenal di trainset

        seen_books = set([isbn for (u, isbn) in train_data if u == user])
        unseen_books = [isbn for isbn in all_isbns if isbn not in seen_books]

        predictions = [model.predict(user, isbn) for isbn in unseen_books]
        predictions.sort(key=lambda x: x.est, reverse=True)

        top_n[user] = [pred.iid for pred in predictions[:n]]

    return top_n

# ISBN unik
all_isbns = df2["isbn"].unique()
users_to_eval = list(test_ground_truth.keys())

# Buat prediksi rekomendasi
predictions_cf = get_top_n_cf(model_cf, trainset_surp_cf, all_isbns, users_to_eval, n=10)
print("Predictions (CF): ", predictions_cf)
print("Target Ground Truth:", test_ground_truth)

Predictions (CF):  defaultdict(<class 'list'>, {7346: ['0452264464', '0971880107', '0345402871', '0345417623', '0446310786', '0842342702', '0440225701', '042511774X', '0316769487', '0679429220'], 11676: ['0345402871', '0425182908', '0679429220', '0671867156', '0060976845', '0671004573', '038572179X', '0330332775', '0060977493', '0316096199'], 13552: ['0452264464', '0345402871', '0345417623', '0671888587', '0842342702', '042511774X', '0804106304', '0140067477', '0345465083', '0316769487'], 30533: ['0452264464', '0449005615', '0553582747', '0842342702', '0440225701', '042511774X', '0140067477', '0451166892', '0786868716', '0446612545'], 31315: ['0440234743', '0452264464', '0971880107', '0345402871', '0345417623', '0446310786', '0553582747', '0425182908', '0842342702', '0140067477'], 78973: ['0452264464', '0971880107', '0345417623', '0449005615', '0425182908', '0140067477', '0316769487', '0679429220', '0671867156', '0451166892'], 101209: ['0971880107', '0345417623', '0446310786', '0440225

**Hasil:** `predictions_cf`: berisi top-10 rekomendasi buku untuk tiap pengguna berdasarkan model CF. Rekomendasi akan dibandingkan dengan `test_ground_truth` untuk dapat skor evaluasi.

## Evaluation

### Metrik Precision & Recall

In [37]:
# Evaluasi
def evaluate_precision_recall(predictions, ground_truth):
    precisions, recalls = [], []

    for user in ground_truth:
        if user not in predictions:
            continue
        true_items = set(ground_truth[user])
        pred_items = set(predictions[user])
        tp = len(true_items & pred_items)

        precision = tp / len(pred_items) if pred_items else 0
        recall = tp / len(true_items) if true_items else 0

        precisions.append(precision)
        recalls.append(recall)

    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    return avg_precision, avg_recall

### Content-Based Filtering (CBF)
Evaluasi dengan precision & recall

In [38]:
precision_cbf, recall_cbf = evaluate_precision_recall(predictions_cbf, test_ground_truth) # bandingkan hasil prediksi dan target
print("Predictions:", predictions_cbf)
print("Ground Truth:", test_ground_truth)
print(f"Precision (CBF): {precision_cbf:.4f}, Recall (CBF): {recall_cbf:.4f}")

Predictions: {7346: ['044022165X', '0440241537', '0440211727', '0440220602', '0440241073', '0804114986', '080411109X', '080410753X', '0671021001', '0446672211'], 11676: ['044022165X', '0440241537', '0440211727', '0440220602', '0440241073', '0842329129', '0842329218', '0842329242', '0425167313', '0451166892'], 13552: ['0385484518', '0671004565', '0553582755', '0786885688', '0515132020', '0440220602', '0440241073', '0440213525', '044023722X', '044021145X'], 30533: ['0380789019', '0380789035', '1573229326', '0441569595', '0743418174', '0375707972', '0679745203', '0375703861', '0380002930', '0375702709'], 31315: ['0446612545', '0375705856', '0553273914', '038550120X', '0375702709', '0553292722', '0553561618', '055356451X', '0553579606', '0553571885'], 78973: ['0553579754', '0553580221', '0553582755', '042513525X', '0345386108', '0670892963', '014028009X', '0141000198', '0330332775', '0140119906'], 101209: ['044022165X', '0440241537', '0440211727', '0440220602', '0440241073', '0375725784', 

**Interpretasi:**
- `Precision = 0.0221 (≈ 2.21%)` Dari seluruh buku yang direkomendasikan, hanya 2.21% yang benar-benar disukai oleh user.
- `Recall = 0.0632 (≈ 6.32%)` Dari semua buku yang seharusnya direkomendasikan (user_liked_books), hanya 6.32% yang berhasil diprediksi.
- Precision tinggi → model merekomendasikan buku yang benar-benar disukai pengguna.
- Recall tinggi → model berhasil menangkap sebagian besar buku yang disukai pengguna.
- Jika kedua nilai rendah, kemungkinan:
 - Deskripsi buku tidak cukup informatif.
 - Kesamaan konten (TF-IDF) tidak mencerminkan preferensi pengguna.
 - Perlu pendekatan lain: collaborative filtering, matrix factorization, dll.

### Collaborative Filtering (CF)
Evaluasi precision dan recall

In [40]:
precision_cf, recall_cf = evaluate_precision_recall(predictions_cf, test_ground_truth) # bandingkan hasil prediksi dan target
print("Predictions:", predictions_cf)
print("Ground Truth:", test_ground_truth)
print(f"Precision (CF): {precision_cf:.4f}, Recall (CF): {recall_cf:.4f}")

Predictions: defaultdict(<class 'list'>, {7346: ['0452264464', '0971880107', '0345402871', '0345417623', '0446310786', '0842342702', '0440225701', '042511774X', '0316769487', '0679429220'], 11676: ['0345402871', '0425182908', '0679429220', '0671867156', '0060976845', '0671004573', '038572179X', '0330332775', '0060977493', '0316096199'], 13552: ['0452264464', '0345402871', '0345417623', '0671888587', '0842342702', '042511774X', '0804106304', '0140067477', '0345465083', '0316769487'], 30533: ['0452264464', '0449005615', '0553582747', '0842342702', '0440225701', '042511774X', '0140067477', '0451166892', '0786868716', '0446612545'], 31315: ['0440234743', '0452264464', '0971880107', '0345402871', '0345417623', '0446310786', '0553582747', '0425182908', '0842342702', '0140067477'], 78973: ['0452264464', '0971880107', '0345417623', '0449005615', '0425182908', '0140067477', '0316769487', '0679429220', '0671867156', '0451166892'], 101209: ['0971880107', '0345417623', '0446310786', '0440225701', 

**Inferensi:**
- `Precision = 0.0114 (~1.14%)` Dari seluruh rekomendasi yang diberikan oleh model ke pengguna, hanya 1.28% yang benar-benar sesuai dengan selera mereka (buku dengan rating tertinggi).
- `Recall = 0.0367 (~3.67%)` Dari semua buku favorit (yang pengguna beri rating tertinggi), hanya 3.67% yang berhasil direkomendasikan oleh model.

### Perbandingan Hasil Metrik

| Model                     | Precision | Recall  |
|--------------------------|-----------|---------|
| Content-Based Filtering  | 0.0221    | 0.0632  |
| Collaborative Filtering  | 0.0114    | 0.0367  |

- CBF unggul dalam precision dan recall dibandingkan CF.
- CBF lebih baik dalam merekomendasikan buku yang benar-benar disukai (rating tinggi).
- Namun, kedua model masih memiliki akurasi rendah secara keseluruhan, yang menunjukkan perlunya peningkatan atau pendekatan hybrid.

### Too-10 Rekomendasi
Mengambil data user random dan kedua model akan mengembalikan 10 Rekomendasi terbaik bukunya masing masing. Penjelasan Kode:
- `df_book[df_book['isbn'].isin(cbf_isbns)]` mengambil baris yang ISBN-nya ada dalam daftar rekomendasi. Hal yang sama unutk CF
- `.loc[cbf_isbns]` menjaga urutan sesuai urutan rekomendasi. Hal yang sama unutk CF
- Ditampilkan dalam format tabel dengan kolom isbn dan book_title.

In [28]:
# Pilih satu user secara acak dari test_ground_truth
# sample_user = random.choice(list(test_ground_truth.keys())) # Versi random
sample_user = 261105
user_gt_isbns = test_ground_truth[sample_user]

# Ambil judul buku berdasarkan ISBN
gt_titles = df_book[df_book['isbn'].isin(user_gt_isbns)][['isbn', 'book_title']]

# Tampilkan hasil
print(f"\nUser: {sample_user}")
print(f"Ground Truth (buku paling disukai):")
display(gt_titles.reset_index(drop=True))

# Ambil daftar ISBN rekomendasi
cbf_isbns = predictions_cbf.get(sample_user, [])
cf_isbns = predictions_cf.get(sample_user, [])

# Buat DataFrame hasil rekomendasi untuk CBF
cbf_df = df_book[df_book['isbn'].isin(cbf_isbns)][['isbn', 'book_title']]
cbf_df = cbf_df.set_index('isbn').loc[cbf_isbns].reset_index()
print("\nTop-10 Rekomendasi (Content-Based Filtering):")
display(cbf_df)

# Buat DataFrame hasil rekomendasi untuk CF
cf_df = df_book[df_book['isbn'].isin(cf_isbns)][['isbn', 'book_title']]
cf_df = cf_df.set_index('isbn').loc[cf_isbns].reset_index()
print("\nTop-10 Rekomendasi (Collaborative Filtering):")
display(cf_df)


User: 261105
Ground Truth (buku paling disukai):


,isbn,book_title
0,0345337662,Interview with the Vampire



Top-10 Rekomendasi (Content-Based Filtering):


,isbn,book_title
0,0515127833,River's End
1,0743227441,The Other Boleyn Girl
2,0380731851,Mystic River
3,0316899984,"River, Cross My Heart"
4,0743203631,Gap Creek: The Story Of A Marriage
5,044022165X,The Rainmaker
6,0440234743,The Testament
7,0440241537,The King of Torts
8,0440211727,A Time to Kill
9,0440220602,The Chamber



Top-10 Rekomendasi (Collaborative Filtering):


,isbn,book_title
0,0440234743,The Testament
1,0452264464,Beloved (Plume Contemporary Fiction)
2,0345402871,Airframe
3,0446310786,To Kill a Mockingbird
4,0671888587,I'll Be Seeing You
5,0553582747,From the Corner of His Eye
6,0440225701,The Street Lawyer
7,0140067477,The Tao of Pooh
8,0345465083,Seabiscuit
9,0679429220,Midnight in the Garden of Good and Evil: A Savannah Story


Hasilnya:
- User: 261105
- Buku yang disukai (Ground Truth): '0345337662' → *Interview with the Vampire*
- CBF cenderung merekomendasikan buku dengan kemiripan deskriptif terhadap buku favorit user.
- CF memberikan buku yang disukai oleh pengguna lain dengan preferensi yang mirip, sehingga hasil lebih beragam secara konten.
- Beberapa buku seperti The Testament muncul di kedua sistem, menandakan bahwa buku itu relevan secara konten dan populer di kalangan pengguna serupa.

